In [ ]:
from configparser import ConfigParser
import spotipy
import sys
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
import numpy as np

parser = ConfigParser()
parser.read('./spotify_credentials.cfg')

SPOTIPY_CLIENT_ID = parser.get('spotify', 'SPOTIPY_CLIENT_ID')
SPOTIPY_CLIENT_SECRET = parser.get('spotify', 'SPOTIPY_CLIENT_SECRET')

sp = spotipy.Spotify(client_credentials_manager=SpotifyClientCredentials(client_id=SPOTIPY_CLIENT_ID, client_secret=SPOTIPY_CLIENT_SECRET))

In [ ]:
input = pd.read_csv('artists.csv', header = None)
artist_ids = []
for link in input[0]:
    
    artist_ids.append(link[32:54])

artist_ids

In [ ]:
albums = []
album_track_ids = []
album_ids = []
artist_uri = 'spotify:artist:' + artist_ids[0]

# Add album ids to album_ids list
results = sp.artist_albums(artist_uri, album_type='album')
albums.extend(results['items'])
while results['next']:
    results = sp.next(results)
    albums.extend(results['items'])
for album in albums:
    album_ids.append(album['id'])

all_tracks = []
for id in album_ids:

    track_ids = []

    results = sp.album_tracks(id)
    tracks = results['items']
    all_tracks.extend(tracks)
    for track in tracks:
        track_ids.append(track['id'])

    album_track_ids.append(track_ids)


assert len(album_track_ids) == len(albums)

len(albums)

In [ ]:
albums = albums[0:2]
albums

In [11]:
# sp.audio_features('2RpXUwCytDFqZSHl6kIquB')
sp.audio_analysis('2RpXUwCytDFqZSHl6kIquB')['segments']

[{'start': 0.0,
  'duration': 1.05673,
  'confidence': 0.232,
  'loudness_start': -60.0,
  'loudness_max_time': 0.95428,
  'loudness_max': -55.043,
  'loudness_end': 0.0,
  'pitches': [0.558,
   0.469,
   0.561,
   0.526,
   0.709,
   0.811,
   0.703,
   0.967,
   1.0,
   0.815,
   0.774,
   0.568],
  'timbre': [0.469,
   174.268,
   20.462,
   -41.16,
   54.195,
   -56.226,
   16.723,
   2.317,
   -22.811,
   2.455,
   4.417,
   -9.233]},
 {'start': 1.05673,
  'duration': 0.15025,
  'confidence': 0.709,
  'loudness_start': -55.396,
  'loudness_max_time': 0.10679,
  'loudness_max': -47.577,
  'loudness_end': 0.0,
  'pitches': [0.306,
   0.377,
   0.273,
   0.311,
   0.417,
   0.572,
   0.548,
   0.632,
   1.0,
   0.602,
   0.346,
   0.344],
  'timbre': [10.373,
   215.295,
   109.167,
   -14.684,
   60.357,
   -11.126,
   14.345,
   -6.295,
   44.845,
   13.446,
   4.572,
   -5.145]},
 {'start': 1.20698,
  'duration': 0.28494,
  'confidence': 0.302,
  'loudness_start': -48.616,
  'loud

In [ ]:
# columns_to_remove = ['analysis_url', 'type', 'uri', 'track_href']
headings = ['start',
'duration',
'confidence',
'loudness',
'tempo',
'tempo_confidence',
'key',
'key_confidence',
'mode',
'mode_confidence',
'time_signature',
'time_signature_confidence']
data = pd.DataFrame(columns=headings)

# Create DataFrames of features from track ids
i = 0
j = 0
for album in album_track_ids:
    
    length = len(album)
    
    for track in album:
        track_sections = sp.audio_analysis(track)['sections']
        for results in track_sections:
            if results: 
                features = results
            features_matrix = pd.DataFrame.from_records(features, index=[0])
            
            # Remove unneeded columns
            # features_matrix.drop(columns = columns_to_remove, axis = 1, inplace = True)
            # print(features_matrix)

            # Add track title
            track_title = all_tracks[j]['name']
            features_matrix['track_title'] = track_title
            # Add album title
            album_title = albums[i]['name']
            features_matrix['album_title'] = album_title
            # Add artist name
            name = albums[i]['artists'][0]['name']
            features_matrix['album_artist'] = name

            # Add track number and total tracks
            features_matrix['track_number'] = all_tracks[j]['track_number']
            features_matrix['total_tracks'] = length

            # Add album as a row to the data df
            # print(features_matrix,'\n')
            # print(data)
            data = data.append(features_matrix)

        j += 1

    i += 1

# Remove duplicates
data.drop_duplicates(subset=['track_number', 'album_title', 'album_artist'])
data

In [ ]:
data['order'] = data['track_number']/data['total_tracks']

data.to_csv('./data/sections.csv')
data

In [ ]:
# columns_to_remove = ['analysis_url', 'type', 'uri', 'track_href']
headings = ['Start','Duration', 'Confidence', 'Loudness_Start',
            'Loudness_Max_Time', 'Loudness_Max', 'Loudness_Max',
            'Loudness_End', 'Pitches', 'Timbres']
data = pd.DataFrame(columns=headings)

# Create DataFrames of features from track ids
i = 0
j = 0
k = 0
pitches = [0] * len(all_tracks*1500)
timbres = [0] * len(all_tracks*1500)
for album in album_track_ids:
    
    length = len(album)
    
    for track in album:
        track_segments = sp.audio_analysis(track)['segments']
        for result in track_segments:
            pitches[k] = result.get('pitches')
            del result['pitches']
            timbres[k] = result.get('timbre')
            del result['timbre']
            k += 1
        for results in track_segments:
            if results: 
                features = results
            features_matrix = pd.DataFrame.from_records(features, index=[0])
            
            # Remove unneeded columns
            # features_matrix.drop(columns = columns_to_remove, axis = 1, inplace = True)
            # print(features_matrix)

            # Add track title
            track_title = all_tracks[j]['name']
            features_matrix['track_title'] = track_title
            # Add album title
            album_title = albums[i]['name']
            features_matrix['album_title'] = album_title
            # Add artist name
            name = albums[i]['artists'][0]['name']
            features_matrix['album_artist'] = name

            # Add track number and total tracks
            features_matrix['track_number'] = all_tracks[j]['track_number']
            features_matrix['total_tracks'] = length

            # Add album as a row to the data df
            # print(features_matrix,'\n')
            # print(data)
            data = data.append(features_matrix)

        j += 1

    i += 1

# Remove duplicates
data.drop_duplicates(subset=['track_number', 'album_title', 'album_artist'])
data

In [ ]:
data['pitches'] = pitches
data['timbre'] = timbres
data['order'] = data['track_number']/data['total_tracks']

data.to_csv('./data/segments.csv')
data

In [30]:
# data = np.zeros((len(all_tracks*1500),9))
data = np.zeros((1,10))
titles = ['Start','Duration', 'Confidence', 'Loudness_Start',
            'Loudness_Max_Time', 'Loudness_Max', 'Loudness_Max',
            'Loudness_End', 'Pitches', 'Timbres']
result = sp.audio_analysis('2RpXUwCytDFqZSHl6kIquB')['segments']
data[0] = result[0].values()

temp = result[0].values()
temp

TypeError: float() argument must be a string or a number, not 'dict_values'